In [3]:
pip install psycopg2

  Using cached psycopg2-2.9.9-cp312-cp312-macosx_14_0_arm64.whl
Note: you may need to restart the kernel to use updated packages.


# Indigo Flight Data in Database

In [6]:
import http.client
import json
import psycopg2
from datetime import datetime

# Step 1: Fetch Data from the API
conn = http.client.HTTPSConnection("tripadvisor16.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "939577bcd4msh51c844c189119f7p129984jsnf3e058bd1f56",  # Replace with your actual API key
    'x-rapidapi-host': "tripadvisor16.p.rapidapi.com"
}

# Make API request
conn.request("GET", "/api/v1/flights/searchFlights?sourceAirportCode=BOM&destinationAirportCode=DEL&date=2024-10-17&itineraryType=ONE_WAY&sortOrder=ML_BEST_VALUE&numAdults=1&numSeniors=0&classOfService=ECONOMY&pageNumber=1&nearby=yes&nonstop=yes&currencyCode=INR&airlines=6E", headers=headers)

res = conn.getresponse()
data = res.read()
decoded_data = data.decode("utf-8")

# Parse JSON response
response = json.loads(decoded_data)

# Step 2: Connect to PostgreSQL
conn = psycopg2.connect("dbname=TransitGuide user=postgres password=0000")
cur = conn.cursor()

# SQL Insert Query
insert_query = """
    INSERT INTO IndigoFlightsTable (source, destination, flight_number, departure_date, departure_time, arrival_time, fare, class)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

# Step 3: Parse and Insert Data
flights = response.get('data', {}).get('flights', [])

for flight in flights:
    for segment in flight['segments']:
        for leg in segment['legs']:
            source = leg['originStationCode']
            destination = leg['destinationStationCode']
            flight_number = leg['flightNumber']
            
            # Convert departure and arrival to separate date and time
            departure_datetime = datetime.fromisoformat(leg['departureDateTime'])
            arrival_datetime = datetime.fromisoformat(leg['arrivalDateTime'])
            departure_date = departure_datetime.date()
            departure_time = departure_datetime.time()
            arrival_time = arrival_datetime.time()
            
            # Get fare
            fare = flight['purchaseLinks'][0]['totalPricePerPassenger']  # Get fare from the first purchase link
            class_of_service = leg['classOfService']
            
            # Insert into database
            cur.execute(insert_query, (
                source, destination, flight_number, 
                departure_date, departure_time, 
                arrival_time, fare, class_of_service
            ))

# Commit the transaction
conn.commit()

# Close connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [7]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 21.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file_path = '/Users/mbhanu/Downloads/airport_data.csv'
df = pd.read_csv(csv_file_path)

# Connect to PostgreSQL
conn = psycopg2.connect("dbname=TransitGuide user=postgres password=0000")
cur = conn.cursor()


# Insert the data into the AirportData table
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO AirportData (type, name, latitude_deg, longitude_deg, city, airport_code)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
    cur.execute(insert_query, (row['type'], row['name'], row['latitude_deg'], row['longitude_deg'], row['city'], row['airport_code']))

# Commit the transaction
conn.commit()

# Close connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [9]:
pip install flask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.
